# Homework 2 Evolutionary Algorithms.
This file will include all the necessary code for evolutionary algorithm homework. The task is described as "In this homework, you will perform experiments on evolutionary algorithm and draw conclusions from the experimental results. The task is to create an image made of filled circles, visually similar to a given RGB source image (painting.png)"

## Pseudo code
Initialize population with <num_inds> individuals each having <num_genes> genes
While not all generations (<num_generations>) are computed:
Evaluate all the individuals
Select individuals
Do crossover on some individuals
Mutate some individuals

In [1]:
# "Individual" class definition for evolutionary algoritm. There will be on chromosome and N number of genes. Each gene will have center coordinates (x,y), Radius, and RGB color.

import random
import math
import numpy as np
import cv2

class Individual:
    def __init__(self, num_genes, image_size):
        self.num_genes = num_genes
        self.image_size = image_size
        self.chromosome = []
        self.fitness = 0
        self.elite = False

        for i in range(num_genes):
            x = random.randint(0, image_size[0])
            y = random.randint(0, image_size[1])
            r = random.randint(0, 50)
            color = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
            alpha = random.random()

            self.chromosome.append([x, y, r, color, alpha])

    def mutate(self, mutation_probability, guidance = None):
        if not self.elite:
            if guidance is None:
                for i in range(self.num_genes):
                    if random.random() < mutation_probability:
                        self.chromosome[i][0] = random.randint(0, self.image_size[0])
                        self.chromosome[i][1] = random.randint(0, self.image_size[1])
                        self.chromosome[i][2] = random.randint(0, 50)
                        self.chromosome[i][3] = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
            else:
                #Guided mutation, deviate x,y, radius, color and alpha around their previous values
                for i in range(self.num_genes):
                    if random.random() < mutation_probability:
                        self.chromosome[i][0] = int(self.chromosome[i][0] + (self.image_size[0]/2)*random.uniform(-1,1))
                        self.chromosome[i][1] = int(self.chromosome[i][1] + (self.image_size[1]/2)*random.uniform(-1,1))
                        self.chromosome[i][2] = int(self.chromosome[i][2] + 10*random.uniform(-1,1))
                        self.chromosome[i][3][0] = self.chromosome[i][3][0] + 64*random.uniform(-1, 1)
                        if self.chromosome[i][3][0] < 0:
                            self.chromosome[i][3][0] = 0
                        elif self.chromosome[i][3][0] > 255:
                            self.chromosome[i][3][0] = 255

                        self.chromosome[i][3][1] = self.chromosome[i][3][1] + 64*random.uniform(-1, 1)
                        if self.chromosome[i][3][1] < 0:
                            self.chromosome[i][3][1] = 0
                        elif self.chromosome[i][3][1] > 255:
                            self.chromosome[i][3][1] = 255
                        
                        self.chromosome[i][3][2] = self.chromosome[i][3][2] + 64*random.uniform(-1, 1)
                        if self.chromosome[i][3][2] < 0:
                            self.chromosome[i][3][2] = 0
                        elif self.chromosome[i][3][2] > 255:
                            self.chromosome[i][3][2] = 255

                        self.chromosome[i][3] = [int(x) for x in self.chromosome[i][3]]
                        
                        self.chromosome[i][4] = self.chromosome[i][4] + 0.25*random.uniform(-1, 1)
                        if self.chromosome[i][4] < 0:
                            self.chromosome[i][4] = 0
                        elif self.chromosome[i][4] > 1:
                            self.chromosome[i][4] = 1

                        # #Check if the gene is still in the image
                        # if self.chromosome[i][0] - self.chromosome[i][2] < 0 or self.chromosome[i][0] + self.chromosome[i][2] > self.image_size[0] or self.chromosome[i][1] - self.chromosome[i][2] < 0 or self.chromosome[i][1] + self.chromosome[i][2] > self.image_size[1]:
                        #     # randomly initialize the gene and check again
                        #     self.chromosome[i][0] = random.randint(0, self.image_size[0])
                        #     self.chromosome[i][1] = random.randint(0, self.image_size[1])
                        #     self.chromosome[i][2] = random.randint(0, 50)
    def draw(self):
        # First sort the genes by radius
        self.chromosome.sort(key=lambda x: x[2])


        image = np.zeros((self.image_size[1], self.image_size[0], 3), np.uint8)
        for gene in self.chromosome:
            #check if the circle is visible in the image, center does not have to be in the image but the circle should be visible
            if gene[0] - gene[2] < 0 or gene[0] + gene[2] > self.image_size[0] or gene[1] - gene[2] < 0 or gene[1] + gene[2] > self.image_size[1]:
                # randomly initialize the gene and check again
               
                gene[0] = random.randint(0, self.image_size[0])
                gene[1] = random.randint(0, self.image_size[1])
                gene[2] = random.randint(0, 50)

            cv2.circle(image, (gene[0], gene[1]), gene[2], gene[3], -1)
        return image

    def calculate_fitness(self, target):
        image = self.draw()
        diff = cv2.absdiff(image, target)
        self.fitness = np.sum(diff)

    def crossover(self, partner):
        child = Individual(self.num_genes, self.image_size)
        for i in range(self.num_genes):
            if random.random() < 0.5:
                child.chromosome[i] = self.chromosome[i]
            else:
                child.chromosome[i] = partner.chromosome[i]
        return child

In [ ]:
# Popoulation class definition

class Population:
    def __init__(self, num_individuals, num_genes, image_size, num_elites, num_parents, tm_size):
        self.num_individuals = num_individuals
        self.num_genes = num_genes
        self.image_size = image_size
        self.num_elites = num_elites
        self.num_parents = num_parents
        self.tm_size = tm_size

        self.individuals = []
        self.target = np.zeros((self.image_size[1], self.image_size[0], 3), np.uint8)
        self.target[:] = 255

        for i in range(self.num_individuals):
            self.individuals.append(Individual(self.num_genes, self.image_size))

    def evaluate(self):
        for individual in self.individuals:
            individual.calculate_fitness(self.target)

    def selection(self):
        self.individuals.sort(key=lambda x: x.fitness)
        # Mark the best individuals as elite
        for i in range(self.num_elites):
            self.individuals[i].elite = True

        self.parents = self.individuals[self.num_elites:self.num_parents]

    def crossover(self):
        new_individuals = []
        for i in range(self.num_individuals):
            parent1 = random.choice(self.individuals)
            parent2 = random.choice(self.individuals)
            child = parent1.crossover(parent2)
            new_individuals.append(child)
        self.individuals = new_individuals

    def mutation(self, mutation_probability):
        #check if the individual is an elite, if so do not mutate
        for individual in self.individuals: 
            individual.mutate(mutation_probability)

    def get_best(self):
        self.individuals.sort(key=lambda x: x.fitness)
        return self.individuals[0]

    def get_average_fitness(self):
        return sum([x.fitness for x in self.individuals]) / self.num_individuals